<a href="https://colab.research.google.com/github/FedericoFortuna/ProgramacionConcurrente/blob/main/TP1_P3_C%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Comunicación de procesos en C++ (FIFO)**



In [338]:
%%writefile servidor.cpp
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <fcntl.h>
#include <unistd.h>
#include <sys/stat.h>

#define FIFO_NAME "/tmp/fifo"

int main() 
{
    int fifo;
    int numeros[4] = {0};
    float promedio = 0;
    int min = 0;
    int max = 0;
    int suma = 0;
    int cantidad = 0;
    char buffer[BUFSIZ];

    /* Crea la tubería nombrada */
    if(mkfifo(FIFO_NAME, 0666) < 0) 
    {
        printf("No se pudo abrir la tuberia.\n");
        perror("mkfifo");
        return EXIT_FAILURE;
    }

    /* Abre la tubería nombrada */
    if((fifo = open(FIFO_NAME, O_RDWR)) < 0) 
    {
        perror("open");
        return EXIT_FAILURE;
    }
    
    /* Lee la cadena de números desde la tubería nombrada */
    if(read(fifo, buffer, BUFSIZ) < 0) 
    {
        printf("No se pudo leer la tuberia.\n");
        perror("read");
        return EXIT_FAILURE;
    }

    /* Convierte la cadena de números a un arreglo de enteros */
    sscanf(buffer, "%d-%d-%d-%d", &numeros[0], &numeros[1], &numeros[2], &numeros[3]);

    /* Calcula el promedio, mínimo, máximo, suma y cantidad de números */
    for(int i = 0; i < 4; i++) 
    {
        suma += numeros[i];
        cantidad++;
        if(i == 0 || numeros[i] < min) 
        {
            min = numeros[i];
        }
        if(i == 0 || numeros[i] > max) 
        {
            max = numeros[i];
        }
    }
    promedio = (float)suma / cantidad;

    /* Convierte los resultados a una cadena y los escribe en la tubería nombrada */
    sprintf(buffer, "%f,%d,%d,%d,%d", promedio, min, max, suma, cantidad);

    if(write(fifo, buffer, strlen(buffer)) < 0) 
    {
        printf("No se pudo escribir el resultado.\n");
        perror("write");
        close(fifo);
        return EXIT_FAILURE;
    }

    /* Cierra la tubería nombrada */
    close(fifo);

    /* Elimina la tubería nombrada */
    
    if(unlink(FIFO_NAME) < 0) 
    {
        perror("unlink");
        return EXIT_FAILURE;
    }

    return EXIT_SUCCESS;
}



Overwriting servidor.cpp


In [319]:
%%writefile cliente.cpp
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <fcntl.h>
#include <unistd.h>

#define FIFO_NAME "/tmp/fifo"

int main(int argc, char *argv[]) 
{
    int fifo;
    float promedio = 0;
    int min = 0;
    int max = 0;
    int suma = 0;
    int cantidad = 0;
    char buffer[BUFSIZ];

    if(argc < 2) 
    {
        printf("Debe ingresar una cadena con números enteros separados por guión medio.\n");
        return EXIT_FAILURE;
    }

    /* Abre la tubería nombrada */
    if((fifo = open(FIFO_NAME, O_RDWR)) < 0) 
    {
        printf("No se pudo abrir tuberia.\n");
        perror("open");
        return EXIT_FAILURE;
    }

    /* Escribe la cadena de números en la tubería nombrada */
    if(write(fifo, argv[1], strlen(argv[1])) < 0) 
    {
        printf("No se pudo escribir tuberia.\n");
        perror("write");
        return EXIT_FAILURE;
    }

    /* Lee los resultados desde la tubería nombrada */
    if(read(fifo, buffer, BUFSIZ) < 0) 
    {
        printf("No se pudo leer tuberia.\n");
        perror("read");
        return EXIT_FAILURE;
    }

    /* Convierte los resultados de cadena a variables */
    sscanf(buffer, "%f,%d,%d,%d,%d", &promedio, &min, &max, &suma, &cantidad);

    /* Muestra los resultados por pantalla */
    printf("Promedio: %f\n", promedio);
    printf("Mínimo: %d\n", min);
    printf("Máximo: %d\n", max);
    printf("Suma: %d\n", suma);
    printf("Cantidad: %d\n", cantidad);

    /* Cierra la tubería nombrada */
    close(fifo);

    return EXIT_SUCCESS;
}



Overwriting cliente.cpp


In [339]:
!g++ servidor.cpp -o servidor.bin

In [321]:
!g++ cliente.cpp -o cliente.bin

In [340]:
!nohup ./servidor.bin 1>salidaServidor 2>/dev/null &

In [341]:
!./cliente.bin 1-2-3-4 1>salidaCliente 2>/dev/null

In [342]:
!cat salidaCliente

Promedio: 2.500000
Mínimo: 1
Máximo: 4
Suma: 10
Cantidad: 4
